# 6 RAG
우리는 Document GPT App을 만들거다. 하지만 Streamlit을 바로 사용하진 않을거라 조금 투박해보일것임.
Streamlit이 우리에게 멋진 UI를 제공할 것이다.
그래서 우리는 먼저 Jupyter Notebook을 사용할 것이다.
여기서 app의 파트별 흐름을 파악한 뒤에 streamlit을 배울 것이다.
그 뒤에 app을 구성하고, 채팅으로 만들고, 멋진 UI를 입힐 거다.

## 6.0 introduction
이번 섹션에서는 `RAG(Retrieval-Augmented Generation")`을 배울 것이다.
model들은 많은 데이터를 통해 학습 하지만, 개인적인 데이터들엔 때론 접근 할 수 없다.
`개인 DB나 문서 같은 것들`. Model들은 그런 데이터에 대해서 알수 없다.
그래서 우리는 `RAG`을 사용한다.
쉽게 말해 답변에 도움이 되는 정보를 개인으로부터  제공된 data를 사용하거나 탐색하여 Language Model의 능력을 확장하는 것이다.
ChatGPT에서 파일 업로드하여 사용하는 것과 유사?
원한다면 우리가 제공한 정보만으로 답변하게 할 수 있다.
이게 바로 `RAG`!



### Stuff Documents
모든 문서를 prompt에 넣고 model에 전달하는 방식
그 외
* Refine Dcuments chain
* Map Reduce 방식

이건 얼마나 많은 문서를 갖고 있는지, 얼마만큼의 비용으로 운영할지, 어떤 모델을 사용할지, 얼마나 많은 token들을 사용할 수 있는지에 따라 달라진다.


## 6.1 Data Loader and Splitters
RAG의 첫번째  단계에 대해서 알아본다. 바로 Retrival이다.
Retrival은 랭체인의 모듈.

* Source > Load > Transform(Splitting) > Embed > Stroe > Retrieve

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader


# 이제 텍스트를 불러와서 어떻게 되는지 확인해보자!
loader = TextLoader("./files/chapter_one.txt", encoding="utf-8")

loader.load()


[Document(page_content='소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女)딸이라는 걸 알 수 있었다. 소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다. 서울서는 이런 개울물을 보지 못하기나 한 듯 이.\n\n벌써 며칠째 소녀는, 학교에서 돌아오는 길에 물장난이었다. 그런데, 어제까지 개울 기슭에서 하 더니, 오늘은 징검다리 한가운데 앉아서 하고 있다. 소년은 개울둑에 앉아 버렸다. 소녀가 비키기를 기다리자는 것이다. 요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다.\n\n \n\n다음 날은 좀 늦게 개울가로 나왔다.\n\n이 날은 소녀가 징검다리 한가운데 앉아 세수를 하고 있었다. 분홍 스웨터 소매를 걷어올린 목덜미가 마냥 희었다.\n\n한참 세수를 하고 나더니, 이번에는 물 속을 빤히 들여다 본다. 얼굴이라도 비추어 보는 것이리 라. 갑자기 물을 움켜 낸다. 고기 새끼라도 지나가는 듯. 소녀는 소년이 개울둑에 앉아 있는 걸 아는지 모르는지 그냥 날쌔게 물만 움켜 낸다. 그러나, 번번이 허탕이다. 그대로 재미있는 양, 자꾸 물만 움킨다. 어제처럼 개울을 건너는 사람이 있어야 길 을 비킬 모양이다.\n\n그러다가 소녀가 물 속에서 무엇을 하나 집어낸다. 하얀 조약돌이었다. 그리고는 벌떡 일어나 팔짝팔짝 징검다리를 뛰어 건너간다.\n\n다 건너가더니만 홱 이리로 돌아서며, "이 바보."\n\n조약돌이 날아왔다. 소년은 저도 모르게 벌떡 일어섰다.\n\n단발 머리를 나풀거리며 소녀가 막 달린다. 갈밭 사잇길로 들어섰다. 뒤에는 청량한 가을 햇살 아래 빛나는 갈꽃뿐.\n\n이제 저쯤 갈밭머리로 소녀가 나타나리라. 꽤 오랜 시간이 지났다고 생각됐다. 그런데도 소녀는 나타나지 않는다. 발돋움을 했다. 그러고도 상당한 시간이 지났다고 생각됐다.\n\n저 쪽 갈밭머리에 갈꽃이 한 옴큼 움직였다. 소녀가 갈꽃을 안고 있었다. 그리고, 이제는 천천한 걸음이었다. 유난히 맑은 가을 햇살이 소녀의 갈꽃머리에서 반짝거렸다. 소녀 아닌 갈꽃이 

In [16]:
# from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader
# UnstructuredFileLoader는 파일 유형을 고려하지 않아도 되어 편리하다.

loader = UnstructuredFileLoader("./files/chapter_one.docx")
# loader = UnstructuredFileLoader("./files/chapter_one.pdf")
loader.load()


1

이제 문서를 분할 것인데 문서를 분할하는 이유는 
loader.load의 리턴 값을 보면 Document로 이루어진 리스트인데, 
이경우에는 전체 챕터가 하나의 문서에 들어가 있다.

In [17]:
len(loader.load()) # 문서가 하나만 들어있는걸 확인 할 수 있다.

1

문서가 너 무 큰 덩어리이기 때문에, 나누는 작업을 해야한다.
만약 문서를 Embed하거나 저장하거나 언어 모델에 주고 싶다면,,
질문에 답해야할 때 필요한 `파일의 부분들`만을 전달할 수 있다.
그래서 우리는 이 파일을 조각들로 나눠야한다.

* 예를들어, 특정 문자열인 '소녀가 징검다리 한가운데' 부분을 찾고자 한다고 하면,
그러면 이 문자열을 가지고 있는 문서만 언어 모델에 주면 된다.
* 그러기 위해선 챕터 하나를 분할해야한다. 즉 문서의 모든 문자를 조각 내야한다.
이걸 작은 조각들로 나누면, 필요한 걸 찾기 더 쉬워진다.
* 그러면 우리가 만들 prompt도 더 짧아진다.
왜냐하면 전체 문서나 챕터를 LLM에게 주는게 아니라, 딱 필요한 부분만 쓰는거니까.

분할 하는 방법도 여러가지가 있다.
* 우리는 chatGPT3, 4, OpenAI와 잘 작동하는 걸 사용할 것이다.
OpenAI의 실제 함수를 사용해서!
* 우리 텍스트의 길이를 어떤 방식으로 계산해주는 함수

### Text Splitter
아주 일반적인 방법.


In [38]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=200,
#     chunk_overlap=50,
# )
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")


# 첫번쨰 방법
# docs = loader.load()
# splitter.split_documents(docs)

# 다른 방법
loader.load_and_split(text_splitter=splitter)



[Document(page_content='소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女)딸이라는 걸 알 수 있었다. 소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다. 서울서는 이런 개울물을 보지 못하기나 한 듯 이.\n벌써 며칠째 소녀는, 학교에서 돌아오는 길에 물장난이었다. 그런데, 어제까지 개울 기슭에서 하 더니, 오늘은 징검다리 한가운데 앉아서 하고 있다. 소년은 개울둑에 앉아 버렸다. 소녀가 비키기를 기다리자는 것이다. 요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다.\n다음 날은 좀 늦게 개울가로 나왔다.\n이 날은 소녀가 징검다리 한가운데 앉아 세수를 하고 있었다. 분홍 스웨터 소매를 걷어올린 목덜미가 마냥 희었다.\n한참 세수를 하고 나더니, 이번에는 물 속을 빤히 들여다 본다. 얼굴이라도 비추어 보는 것이리 라. 갑자기 물을 움켜 낸다. 고기 새끼라도 지나가는 듯. 소녀는 소년이 개울둑에 앉아 있는 걸 아는지 모르는지 그냥 날쌔게 물만 움켜 낸다. 그러나, 번번이 허탕이다. 그대로 재미있는 양, 자꾸 물만 움킨다. 어제처럼 개울을 건너는 사람이 있어야 길 을 비킬 모양이다.', metadata={'source': './files/chapter_one.docx'}),
 Document(page_content='그러다가 소녀가 물 속에서 무엇을 하나 집어낸다. 하얀 조약돌이었다. 그리고는 벌떡 일어나 팔짝팔짝 징검다리를 뛰어 건너간다.\n다 건너가더니만 홱 이리로 돌아서며, "이 바보."\n조약돌이 날아왔다. 소년은 저도 모르게 벌떡 일어섰다.\n단발 머리를 나풀거리며 소녀가 막 달린다. 갈밭 사잇길로 들어섰다. 뒤에는 청량한 가을 햇살 아래 빛나는 갈꽃뿐.\n이제 저쯤 갈밭머리로 소녀가 나타나리라. 꽤 오랜 시간이 지났다고 생각됐다. 그런데도 소녀는 나타나지 않는다. 발돋움을 했다. 그러고도 상당한 시간이 지났다고 생각됐다.\n저 쪽 갈밭머리에 갈꽃이 한 옴큼 움직였다. 소녀가 갈꽃을 안고 있었다. 그리고, 이제는 천천한

## 6.2 Tiktoken
기본적으로, 이 모든 splitter들은 텍스트의 length를 계산해서, 한 덩어리(chunk) 크기를 알아내서 그 작업에 파이썬 표준 라이브러리가 지원하는 표준 len함수를 사용하지 여기서 나만의 len함수를 명시적으로 설정할 수 있다.
```python
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
    length_function=len,
)
```

In [ ]:
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")


# 첫번쨰 방법
# docs = loader.load()
# splitter.split_documents(docs)

# 다른 방법
loader.load_and_split(text_splitter=splitter)




## 6.3 Vectors

### Embedding에 대해 알아보자.
쉽게 말하면 사람이 읽는 텍스트를 `컴퓨터가 이해할 수 있는 숫자들로 변환하는 작업`이다.

### Vetor
정확히는 vectorization(벡터화) 작업을 할 것이다.
우리가 만든 문서마다 각각의 벡터를 만들어 줄 것이다.
나중에 위에서 split처리를 해준 문서마다 embed작업을 할 것이다.
n 차원 Vector : 추천 시스템에 적합. >>chatGPT에서는 우리가 원하는 것을 찾아주기 위해서 사용할 것이다.


## 6.4 Vector Store

In [60]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore



splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir 
)


vectorstore = Chroma.from_documents(docs, cached_embeddings)


In [61]:
results = vectorstore.similarity_search("소년은 개울가에서 무엇을 했어?")
results

[Document(page_content='소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女)딸이라는 걸 알 수 있었다. 소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다. 서울서는 이런 개울물을 보지 못하기나 한 듯 이.\n벌써 며칠째 소녀는, 학교에서 돌아오는 길에 물장난이었다. 그런데, 어제까지 개울 기슭에서 하 더니, 오늘은 징검다리 한가운데 앉아서 하고 있다. 소년은 개울둑에 앉아 버렸다. 소녀가 비키기를 기다리자는 것이다. 요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다.\n다음 날은 좀 늦게 개울가로 나왔다.', metadata={'source': './files/chapter_one.docx'}),
 Document(page_content='소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女)딸이라는 걸 알 수 있었다. 소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다. 서울서는 이런 개울물을 보지 못하기나 한 듯 이.\n벌써 며칠째 소녀는, 학교에서 돌아오는 길에 물장난이었다. 그런데, 어제까지 개울 기슭에서 하 더니, 오늘은 징검다리 한가운데 앉아서 하고 있다. 소년은 개울둑에 앉아 버렸다. 소녀가 비키기를 기다리자는 것이다. 요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다.\n다음 날은 좀 늦게 개울가로 나왔다.', metadata={'source': './files/chapter_one.docx'}),
 Document(page_content='소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女)딸이라는 걸 알 수 있었다. 소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다. 서울서는 이런 개울물을 보지 못하기나 한 듯 이.\n벌써 며칠째 소녀는, 학교에서 돌아오는 길에 물장난이었다. 그런데, 어제까지 개울 기슭에서 하 더니, 오늘은 징검다리 한가운데 앉아서 하고 있다. 소년은 개울둑에 앉아 버렸다. 소녀가 비키기를 기다리자는 것이다. 요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다.\

## 6.5 Langsmith
Langsmith설정 방법


## 6.6 RetrievalQA

마침내 우리는 document를 이용해 LLM에게 질문해 볼 시간이 되었다!
지금은 off-the-shelf chain들을 사용할 것이다.
우리만의 chain을 LangChain Expression Language로 만들거다.


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
# from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI(temperature=0.1)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir 
)


vectorstore = Chroma.from_documents(docs, cached_embeddings)
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    # chain_type="refine",
    # chain_type="stuff",
    retriever=vectorstore.as_retriever()
)
chain.run("소년은 개울가에 무엇을 했지?")

d:\JUNGWON\Developments\Study\Python\fullstack-gpt\env\Lib\site-packages\langchain\chains\llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'소년은 개울둑에 앉아 버렸다. 소녀가 비키기를 기다리고 있었다.'

### 그외 chain_type
#### stuff
내가 찾은 document들로 prompt를 stuff(채우기)하는데 사용한다는 뜻..
#### Refine (정제, 가다듬기)
query질문을 통해 관련된 document를 얻어서 각각의 document를 읽으면서, 질문에 대한 답변 생성을 시도.
그 작업을 반복하면서 만나는 모든 document를 통해  question을 개선 시킨다.
`요금이 비싸다.`
document가 10개 있다만 각각 10개의 개별 질의를 통하기 때문이다.
#### Map reduce documents chain
query를 입력하면, documents들을 얻어서 각각에 대한 요약 작업을 하게된다.
그리고  각각의 요약본을 LLM에게 전달한다.
굉장히 크고 많은 연산(operation)들이 수행된다.

#### Map re-rank
query시 각 document를 통해 답변을 생성하고, `각 답변에 점수를 매긴다`.
최종적으로` 가장 높은 점수를 획득한 답변과 그 점수를 함께 반환`해준다.

#### RetrivalQA를 사용할떄의 장점 중 하나
* chain_type을 손쉽게 바꿀수 있다.
* Chroma가 나의 컴퓨터에서 실행되는 vector store라는 점이다.
원한다면 `FAISS`도 사용할 수 있다. 이유는 모르겠지만 가끔 이게 더 좋은 성능을 보일때도 있다....


# 6.7 Recap
이떄까지 배운 것을 복습해보자.
Stuff Cahin, MapReduce Chain을 LCCE로 직접 만들어보자.
* 이번 섹션 요약
    * `file load`
        * text loader, `UnstructuredFileLoader`에 대해 살펴보았다. 이걸로 다양한 file들을 load할 수 있다.
    txt, doc, pdf, html, excel 등등...
    * file 분할 (`split`)
        * 아주 긴 text를 다루기 위함. 또 그 긴 text를 작은 document들로 나누고자 했다.
    거대한 단일 document보다는, `작은 여러개를 LLM에게 전달할 때 검색 성능이 좋아지기 때문`이다.
    일단 `저렴`해진다.! document를 작게 분할하면, `작업이 쉬워`지고 `응답도 빨라`지고, LLM 사용 `비용도 훨씬 저렴`해진다.
    * `embedding`
        * text에 `의미별로 적절한 점수를 부여`해서 `vector 형식으로 표현`한 것.
    그리고 `CacheBackedEmbedding`을 사용하여 만들어진 Embedding을 `cache(저장)`했다.
    OpenAIEmbeddings model을 사용했었다.(공짜가 아니다.) 
    * `RetrievalQA`
        * RetrievalQA라는 chain을 이용. 필요한 입력값은 LLM, chain의 종류의 chain_type, 그리고 retriever였다.
    Retriever는 LangChain이 제공하는 class 또는 interface의 일종인데, document를 검색해서 찾아오는(retrieve) 기능을 가지고 있다. 많은 것들이 retriver가 될 수있다. vector store를 이용해서도 만들 수 있다.
    as_retriever 메서드 호출만으로 vector store를 retriever로 전활할 수 있었다.
    


## 6.8 Stuff LCEL Chain
이번 강의에서는 LangChain Expression Language(LCEL)를 사용해서 stuff chain을 구현해볼 것이다.


In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate

from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir 
)


vectorstore = Chroma.from_documents(docs, cached_embeddings)
retriver = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the follwing context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}" ),
    ("human", "{question}")
])
# RunnablePassthrough는 입력값을 말그대로 패스스루 시켜준다...
chian = {"context": retriver, "question": RunnablePassthrough()} | prompt | llm

chain.invoke("소녀은 개울가에 무엇을 했지?")

d:\JUNGWON\Developments\Study\Python\fullstack-gpt\env\Lib\site-packages\langchain\chains\llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'query': '소녀은 개울가에 무엇을 했지?', 'result': '물장난을 하고 있었다.'}

## 6.9 Map Reduce LCEL Chain
이번 강의에서는 LCCE를 이용해서 Map Reduce chain을 직접 구현해볼 것이다.

### 원리
* 일단 document list를 얻어야한다.
* list 내부의 모든 document들을 위한 prompt를 만든다.
    * 이 propmt는 LLM에게 전달할 건데, 기본적인 내용은 다음과 같다.
        * '이 document를 읽고, 사용자의 질문에 답변하기에 적절한 정보가 있는지 확인해주세요.'
* 이를 전달받은 LLM은 응답(response)를 출력할 것이다.
* LLM으로 받은 response들을 취합해서 하나의 document를 만들어 낼 것이다.
* 이렇게 만들어진 단 하나의 최종 document가, LLM을 위한 prompt로 전달될 것이다.


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate

from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
# RunnableLamda는 chian과 그 내부 어디에서든 function을 호출할 수 있도록 해준다.

llm = ChatOpenAI(temperature=0.1)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir 
)


vectorstore = Chroma.from_documents(docs, cached_embeddings)
retriver = vectorstore.as_retriever()


# for doc in list of docs | prompt | llm

# for response in list of llms response | put them all together
map_doc_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """
            Use the following portion of a long document to see if any of the text is relevant to
            answer the question. Return any relevant text verbatim.
            ------
            {context}
        """
    ),
    (
        "human", "{question}"
    )
])

map_doc_chain = map_doc_prompt | llm 

def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    # results = []
    # for document in documents:
    #     result = map_doc_chain.invoke(
    #        { "context": document.page_content,
    #         "question": question }
    #     ).content
    #     results.append(result)
    # results = "\n\n".join(results)
    # return results
    return "\n\n".join(
        map_doc_chain.invoke(
            { "context": doc.page_content, "question": question}
        ).content for doc in documents
    )

map_chain = {"documents": retriver, "question": RunnablePassthrough() } | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        """
        Given the following extracted parts of a long document and a question, create a final answer.
        If you don't know the answer, just say that you don't know. Don't try to make an answer.
        ------
        {context}
        """,
    ),
    ("human", "{question}")

])

chain = {"context": map_chain, "question": RunnablePassthrough() } | final_prompt | llm


chain.invoke("Bana '소년'dan bahset. Türkçe cevap verin.")
# chain.invoke("소녀에 대해서 설명해줘")

AIMessage(content='Bu metindeki "소년" karakteri, bir kızın elini tutarak onu kaldıran ve sonra da ona yardım etmek için bir şeyler yapmaya çalışan biri olarak tanımlanmaktadır.')

## 6.10 Recap
